In [219]:
# %%
import os
from binance.client import Client
import requests
import pandas as pd
from datetime import datetime
import numpy as np

In [220]:
# %%

# Binance
api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')

# Binance Client
client = Client(api_key, api_secret)

startdate = "200 day ago UTC"
# startdate = "10 day ago UTC"
timeframe = "1d"

In [221]:
tickers = client.get_ticker()
dfTickers = pd.DataFrame(tickers)
dfTickers

,symbol,priceChange,priceChangePercent,weightedAvgPrice,prevClosePrice,lastPrice,lastQty,bidPrice,bidQty,askPrice,...,openPrice,highPrice,lowPrice,volume,quoteVolume,openTime,closeTime,firstId,lastId,count
0,ETHBTC,-0.00014900,-0.212,0.06980833,0.07018900,0.07004000,0.41440000,0.07003900,16.36670000,0.07004000,...,0.07018900,0.07077600,0.06853600,142767.30740000,9966.34664522,1668990126729,1669076526729,391505357,391820208,314852
1,LTCBTC,0.00004900,1.277,0.00382217,0.00383800,0.00388600,3.30000000,0.00388600,39.48700000,0.00388700,...,0.00383700,0.00395400,0.00369100,171051.67000000,653.78885227,1668990127677,1669076527677,85702190,85765210,63021
2,BNBBTC,-0.00013400,-0.824,0.01608775,0.01626200,0.01612800,2.66100000,0.01612800,11.62600000,0.01612900,...,0.01626200,0.01629800,0.01580500,79625.98100000,1281.00307680,1668990124735,1669076524735,210456253,210559292,103040
3,NEOBTC,0.00001000,2.513,0.00040100,0.00039800,0.00040800,1.22000000,0.00040800,201.73000000,0.00041000,...,0.00039800,0.00041000,0.00039000,206477.73000000,82.79758616,1668990126232,1669076526232,44045484,44050093,4610
4,QTUMETH,0.00005900,3.333,0.00178508,0.00175300,0.00182900,38.00000000,0.00182200,370.30000000,0.00183600,...,0.00177000,0.00182900,0.00173900,2509.20000000,4.47911230,1668990122565,1669076522565,5340887,5340987,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2124,HFTBUSD,-0.06490000,-12.016,0.46967417,0.54040000,0.47520000,195.30000000,0.47410000,615.80000000,0.47510000,...,0.54010000,0.54390000,0.43000000,32318487.40000000,15179158.88362000,1668990127570,1669076527570,1614247,1669054,54808
2125,HFTUSDT,-0.06590000,-12.177,0.47330674,0.54090000,0.47530000,291.10000000,0.47520000,347.90000000,0.47580000,...,0.54120000,0.54470000,0.43010000,35422982.20000000,16765936.19707000,1668990126824,1669076526824,1959482,2032655,73174
2126,ARPAETH,-0.00000001,-0.044,0.00002389,0.00002270,0.00002282,1022.00000000,0.00002292,9811.00000000,0.00002303,...,0.00002283,0.00002457,0.00002250,2529808.00000000,60.44072379,1668990124561,1669076524561,542,714,173
2127,PHBUSDT,0.26890000,53.121,0.68439362,0.51080000,0.77510000,359.00000000,0.77510000,3722.00000000,0.77880000,...,0.50620000,0.84500000,0.48980000,22035510.00000000,15080962.44870000,1668990127596,1669076527596,338796,423989,85194


In [222]:
exchange_info = client.get_exchange_info()
coinPairs = set()

for s in exchange_info['symbols']:
    if (s['symbol'].endswith('USDT')
        and not(s['symbol'].endswith('DOWNUSDT'))
        and not(s['symbol'].endswith('UPUSDT'))
        and not(s['symbol'].startswith('BUSD'))
        and s['status'] == 'TRADING'):
            coinPairs.add(s['symbol'])
# print(usdt)
coinPairs = sorted(coinPairs)
print(str(len(coinPairs))+" coins found")

325 coins found


In [223]:
# exchange_info['symbols']

In [224]:
def SMA(values, n):
    """Simple moving average"""
    return pd.Series(values).rolling(n).mean()

In [225]:
def applytechnicals(df):
        df['50DSMA'] = df['Close'].rolling(50).mean()
        df['200DSMA'] = df['Close'].rolling(200).mean()

In [226]:
def getdata(Symbol):
    frame = pd.DataFrame(client.get_historical_klines(Symbol,
                                                      timeframe,                                        
                                                      startdate))
    
    frame = frame.iloc[:,[0,4,6]] # columns selection
    frame.columns = ['Time','Close','Volume'] #rename columns
    frame[['Close','Volume']] = frame[['Close','Volume']].astype(float) #cast to float
    # frame.Time = pd.to_datetime(frame.Time, unit='ms') #make human readable timestamp
    frame['Coinpair'] = Symbol
    frame.index = [datetime.fromtimestamp(x/1000.0) for x in frame.Time]
    
    frame = frame[['Coinpair','Close','Volume']]
    return frame

In [227]:
dfResult = pd.DataFrame()
# i = 0

for coinPair in coinPairs:
    # if coinPair == "VGXUSDT":
    #     print(dfResult)
    #     break    
    
    # i = i+1
    # if i == 20:
    #     break

    print("calculating "+coinPair)
    # df = pd.DataFrame()
    # print(len(df.index))
    df = getdata(coinPair)
    applytechnicals(df)
    df.dropna(inplace=True)

    if dfResult.empty:
        dfResult = df
    else:
        dfResult = pd.concat([dfResult, df])


# print(dfResult)


calculating 1INCHUSDT
calculating AAVEUSDT
calculating ACAUSDT
calculating ACHUSDT
calculating ACMUSDT
calculating ADAUSDT
calculating ADXUSDT
calculating AGLDUSDT
calculating AIONUSDT
calculating AKROUSDT
calculating ALCXUSDT
calculating ALGOUSDT
calculating ALICEUSDT
calculating ALPACAUSDT
calculating ALPHAUSDT
calculating ALPINEUSDT
calculating AMPUSDT
calculating ANCUSDT
calculating ANKRUSDT
calculating ANTUSDT
calculating APEUSDT
calculating API3USDT
calculating APTUSDT
calculating ARDRUSDT
calculating ARPAUSDT
calculating ARUSDT
calculating ASRUSDT
calculating ASTRUSDT
calculating ATAUSDT
calculating ATMUSDT
calculating ATOMUSDT
calculating AUCTIONUSDT
calculating AUDIOUSDT
calculating AUDUSDT
calculating AUTOUSDT
calculating AVAUSDT
calculating AVAXUSDT
calculating AXSUSDT
calculating BADGERUSDT
calculating BAKEUSDT
calculating BALUSDT
calculating BANDUSDT
calculating BARUSDT
calculating BATUSDT
calculating BCHUSDT
calculating BEAMUSDT
calculating BELUSDT
calculating BETAUSDT
ca

,Coinpair,Close,Volume,50DSMA,200DSMA
2022-11-22,1INCHUSDT,0.50500,1.669162e+12,0.572380,0.697095
2022-11-22,AAVEUSDT,56.40000,1.669162e+12,74.582000,83.054500
2022-11-22,ACAUSDT,0.09990,1.669162e+12,0.151640,0.255402
2022-11-22,ACHUSDT,0.00918,1.669162e+12,0.011110,0.013433
2022-11-22,ACMUSDT,2.44100,1.669162e+12,3.069280,3.551525
2022-11-22,ADAUSDT,0.30660,1.669162e+12,0.374090,0.467562
2022-11-22,ADXUSDT,0.12070,1.669162e+12,0.153048,0.175934
2022-11-22,AGLDUSDT,0.24100,1.669162e+12,0.301840,0.394685
2022-11-22,AIONUSDT,0.02785,1.669162e+12,0.035622,0.037133
2022-11-22,AKROUSDT,0.00258,1.669162e+12,0.003329,0.004161


In [228]:
conditions = [
    (dfResult['Close'] > dfResult['50DSMA']) & (dfResult['Close'] < dfResult['200DSMA']) & (dfResult['50DSMA'] < dfResult['200DSMA']), # recovery phase
    (dfResult['Close'] > dfResult['50DSMA']) & (dfResult['Close'] > dfResult['200DSMA']) & (dfResult['50DSMA'] < dfResult['200DSMA']), # accumulation phase
    (dfResult['Close'] > dfResult['50DSMA']) & (dfResult['Close'] > dfResult['200DSMA']) & (dfResult['50DSMA'] > dfResult['200DSMA']), # bullish phase
    (dfResult['Close'] < dfResult['50DSMA']) & (dfResult['Close'] > dfResult['200DSMA']) & (dfResult['50DSMA'] > dfResult['200DSMA']), # warning phase
    (dfResult['Close'] < dfResult['50DSMA']) & (dfResult['Close'] < dfResult['200DSMA']) & (dfResult['50DSMA'] > dfResult['200DSMA']), # distribution phase
    (dfResult['Close'] < dfResult['50DSMA']) & (dfResult['Close'] < dfResult['200DSMA']) & (dfResult['50DSMA'] < dfResult['200DSMA'])  # bearish phase
]

values = ['recovery', 'accumulation', 'bullish', 'warning','distribution','bearish']
dfResult['MarketPhase'] = np.select(conditions, values)
# print(dfResult)

,Coinpair,Close,Volume,50DSMA,200DSMA,MarketPhase
2022-11-22,1INCHUSDT,0.50500,1.669162e+12,0.572380,0.697095,bearish
2022-11-22,AAVEUSDT,56.40000,1.669162e+12,74.582000,83.054500,bearish
2022-11-22,ACAUSDT,0.09990,1.669162e+12,0.151640,0.255402,bearish
2022-11-22,ACHUSDT,0.00918,1.669162e+12,0.011110,0.013433,bearish
2022-11-22,ACMUSDT,2.44100,1.669162e+12,3.069280,3.551525,bearish
2022-11-22,ADAUSDT,0.30660,1.669162e+12,0.374090,0.467562,bearish
2022-11-22,ADXUSDT,0.12070,1.669162e+12,0.153048,0.175934,bearish
2022-11-22,AGLDUSDT,0.24100,1.669162e+12,0.301840,0.394685,bearish
2022-11-22,AIONUSDT,0.02785,1.669162e+12,0.035622,0.037133,bearish
2022-11-22,AKROUSDT,0.00258,1.669162e+12,0.003329,0.004161,bearish


In [234]:
dfRecovery = dfResult.query("MarketPhase == 'recovery'")
print("\nCoins in Recovery Market Phase")
print(dfRecovery)

dfAccumulation= dfResult.query("MarketPhase == 'accumulation'")
print("\nCoins in Accumulation Market Phase")
print(dfAccumulation)

dfBullish = dfResult.query("MarketPhase == 'bullish'")
print("\nCoins in Bullish Market Phase")
print(dfBullish)

dfWarning = dfResult.query("MarketPhase == 'warning'")
print("\nCoins in Warning Market Phase")
print(dfWarning)

dfDistribution = dfResult.query("MarketPhase == 'distribution'")
print("\nCoins in Distribution Market Phase")
print(dfDistribution)

dfBearish = dfResult.query("MarketPhase == 'bearish'")
print("\nCoins in Bearish Market Phase")
print(dfBearish)

Coins in Recovery Market Phase
           Coinpair   Close        Volume    50DSMA   200DSMA MarketPhase
2022-11-22  AUDUSDT  0.6621  1.669162e+12  0.645132  0.678778    recovery

Coins in Accumulation Market Phase
           Coinpair  Close        Volume  50DSMA   200DSMA   MarketPhase
2022-11-22  ANTUSDT  1.868  1.669162e+12   1.762  1.774845  accumulation

Coins in Bullish Market Phase
            Coinpair  Close        Volume  50DSMA   200DSMA MarketPhase
2022-11-22  BANDUSDT  1.713  1.669162e+12  1.5256  1.508805     bullish

Coins in Warning Market Phase
Empty DataFrame
Columns: [Coinpair, Close, Volume, 50DSMA, 200DSMA, MarketPhase]
Index: []

Coins in Distribution Market Phase
              Coinpair   Close        Volume     50DSMA    200DSMA  \
2022-11-22  ALPACAUSDT  0.2161  1.669162e+12   0.272392   0.271658   
2022-11-22    ATOMUSDT  9.1250  1.669162e+12  12.076000  11.106060   

             MarketPhase  
2022-11-22  distribution  
2022-11-22  distribution  

Coins in Bear